In [798]:
import pandas as pd

In [799]:
from sklearn.ensemble import RandomForestClassifier

In [800]:
from sklearn.metrics import accuracy_score 

In [801]:
from sklearn.metrics import precision_score

In [802]:
def categorize_performance(mark: int):
  if mark <= 210:
    return 1
  elif mark <= 212:
    return 2
  elif mark <= 215:
    return 3
  
  return 4

In [803]:
def compute_roll_averages(athlete, col, new_col):
  athlete = athlete.sort_values("Date")
  rolling_stats = athlete[col].rolling(2, closed="left").mean()
  athlete[new_col] = rolling_stats
  athlete[new_col] = rolling_stats.dropna()
  return athlete

In [804]:
matches = pd.read_csv("Athletics_rankings.csv")

In [805]:
# Turning the columns of the data into numerical values
matches["Date"] = pd.to_datetime(matches["Date"])
matches["Date_of_Birth"] = pd.to_datetime(matches["Date_of_Birth"])  

C:\Users\gabin\AppData\Local\Temp\ipykernel_11804\349090207.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  matches["Date"] = pd.to_datetime(matches["Date"])


In [806]:
# Add codes to objects in data
matches["Venue_code"] = matches["Venue"].astype("category").cat.codes

In [807]:
matches["Athlete_code"] = matches["Competitor"].astype("category").cat.codes

In [808]:
matches["Day_code"] = matches["Date"].dt.day_of_week

In [809]:
matches["Nationality_code"] = matches["Nationality"].astype("category").cat.codes

In [810]:
matches["Performance_code"] = matches["Mark"].apply(categorize_performance)

In [811]:
# For each athlete, shift "Rank" by 1 year to represent the previous year's rank
matches["Rank_prev_year"] = matches.groupby("Athlete_code")["Rank"].shift(-1).fillna(0)

In [812]:
# Begin training on data that is not linear (number doesn't mean anything just a code for something. Ex. an ID athlete code)
rf = RandomForestClassifier(n_estimators=50, min_samples_split=7, random_state=1) # 7

In [813]:
def make_predictions(data, predictors):
  train = data[(data["Year"] < 2025) & (data["Rank"] <= 15)]
  test = data[(data["Year"] == 2025) & (data["Rank"] <= 15)]

  rf.fit(train[predictors], train["Rank"])
  pred = rf.predict(test[predictors])
  combined = pd.DataFrame(dict(actual=test["Rank"], prediction=pred), index=test.index)
  precision = precision_score(test["Rank"], pred, average="micro")
  return combined, precision
  

In [814]:
predictors = ["Rank", "Athlete_code", "Nationality_code"]

In [815]:
col = ["Mark", "Rank_prev_year"] 


In [816]:
new_col = [f"{c}_rolling" for c in col]

In [817]:
# Computer Rolling Averages for all athletes
athletes_rolling = matches.groupby("Athlete_code").apply(lambda x: compute_roll_averages(x, col, new_col))


C:\Users\gabin\AppData\Local\Temp\ipykernel_11804\2071382099.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  athletes_rolling = matches.groupby("Athlete_code").apply(lambda x: compute_roll_averages(x, col, new_col))


In [818]:
athletes_rolling = athletes_rolling.droplevel("Athlete_code")

In [819]:
combined, precision = make_predictions(athletes_rolling, predictors + new_col)

In [820]:
combined = combined.merge(athletes_rolling[["Athlete_code", "Competitor"]], left_index=True, right_index=True, how="left")

In [821]:
combined

,actual,prediction,Athlete_code,Competitor
13,14,14,21,Anass ESSAYI
10,11,11,23,Andrew COSCORAN
3,4,4,28,Azeddine HABZ
5,6,5,42,Cameron MYERS
11,12,12,82,Ethan STRAND
12,12,12,94,Gary MARTIN
2,3,3,109,Hobbs KESSLER
4,5,4,114,Isaac NADER
0,1,1,124,Jakob INGEBRIGTSEN
14,15,15,143,Josh HOEY


In [822]:
precision

0.7333333333333333